In [2]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel


In [13]:
model_name = "Twitter/twhin-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
LANGUAGE: str = 'English'
GROUPER: str = 'persona'

In [10]:
# Function to encode a single sentence and return its embeddings
def encode_argument(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512, padding='max_length')
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    attention_mask = torch.ones_like(input_ids_tensor)  # Creating attention mask
    with torch.no_grad():
        outputs = model(input_ids_tensor, attention_mask=attention_mask)
        argument_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Mean pooling of token embeddings
    return argument_embedding.tolist()

In [6]:
# Load the arguments from the .parquet file
arguments_df = pd.read_parquet("arguments.by.persona.English.parquet")



In [8]:
arguments_df

,arguments,label
0,"""The US should provide military aid to Ukraine",alt_right
1,"""Russia's actions in Ukraine are a threat to ...",alt_right
2,"""There is a need for a united front from the ...",alt_right
3,"""Congress needs to take action and pass legis...",alt_right
4,"""The US has a role to play in upholding human...",alt_right
...,...,...
138,"""Ukraine needs financial assistance from the ...",neutral
139,"""The conflict in Ukraine is a matter of human...",neutral
140,"""The situation in Ukraine is a test for the i...",neutral
141,"""The West should take a stronger stance again...",neutral


In [11]:
# Create embeddings for each argument and store them in a new column
arguments_df["embedding"] = arguments_df["arguments"].apply(encode_argument)



In [16]:
# Save the DataFrame with embeddings to a new .parquet file
arguments_df.to_parquet(f'argument_embeddings.by.{GROUPER}.{LANGUAGE}.parquet')
arguments_df.to_csv(f'argument_embeddings.by.{GROUPER}.{LANGUAGE}.csv')
